In [1]:
%load_ext autoreload
%autoreload 2

import requests
import msgpack
import zlib
import traceback
import time
import os
import boto3
from concurrent.futures import ThreadPoolExecutor 
import concurrent.futures 
import datetime 
import pandas as pd 
from tqdm.notebook import tqdm
import numpy as np 


from src.model import OrderBookChunksCollection, OrderBook, S3OrderBookDataSource, OrderBooksDataSequenceDatasetV1
import pickle 
PRICE_DIFF_BEST_BINS = pickle.load(open("data/price_diff_best_bins.bin", "rb"))
AMOUNT_BEST_BINS = pickle.load(open("data/amount_usd_best_bins.bin", "rb"))
AMOUNT_INDICES = dict(zip(AMOUNT_BEST_BINS, list(range(len(AMOUNT_BEST_BINS)))))
PRICE_DIFF_INDICES = dict(zip(PRICE_DIFF_BEST_BINS, list(range(len(PRICE_DIFF_BEST_BINS)))))




In [2]:
order_book_col = OrderBookChunksCollection(bucket_name="btc-order-book")

In [3]:
# all_keys = order_book_col.get_all_keys(
#     start_date = datetime.datetime(2019, 10, 27), 
#     end_date = datetime.datetime(2021, 12,22)
# )
# pickle.dump(obj=all_keys, file=open("all_keys.pkl","wb"))
all_keys = pickle.load(open("all_keys.pkl", "rb"))

In [4]:
chunks = OrderBookChunksCollection.get_chunks(keys=all_keys, min_number_of_elements=20000)

In [5]:
len(chunks)

221

In [6]:
k= 0 
for i in range(len(chunks)):
    for chunk in chunks[i].chunks():
            k += 1

In [7]:
k

2325

In [8]:
s3_client = boto3.client("s3")
source = S3OrderBookDataSource(bucket_name="btc-order-book", s3_client=s3_client)

In [ ]:

# t = order_books['2019-10-27/1572188743'].get_tensor(
#     amount_bins=AMOUNT_BEST_BINS,
#     amount_indices=AMOUNT_INDICES, 
#     price_diff_bins=PRICE_DIFF_BEST_BINS,
#     price_diff_indices=PRICE_DIFF_INDICES
# )

In [ ]:
# m = order_books[100].get_sparse_matrices(
#     amount_bins=AMOUNT_BEST_BINS,
#     amount_indices=AMOUNT_INDICES, 
#     price_diff_bins=PRICE_DIFF_BEST_BINS,
#     price_diff_indices=PRICE_DIFF_INDICES
# )

In [ ]:
# m['bids_sparse'].__dir__()

In [ ]:
# from matplotlib import pyplot as plt 
# plt.figure(figsize=(20,20))
# plt.imshow(m['bids_sparse'].toarray(),  cmap='Greys', interpolation='nearest', origin='lower')

In [ ]:
# ts = [] 
# for k in chunks[0].chunks()[0].sorted_keys:
#     ob = order_books[k]
#     t = ob.get_sparse_matrices(
#         amount_bins=AMOUNT_BEST_BINS,
#         amount_indices=AMOUNT_INDICES, 
#         price_diff_bins=PRICE_DIFF_BEST_BINS,
#         price_diff_indices=PRICE_DIFF_INDICES
#     )
#     ts.append(t)
#     print(k)

In [ ]:
# sparse.save_npz(matrix=ts[0]['bids_sparse'], file = "tz0")

In [ ]:
# from scipy import sparse
# sparse.csc_matrix(ts[0][1])

In [ ]:
# t = order_books['2019-10-27/1572188743'].to_tensor(
#     amount_bins=AMOUNT_BEST_BINS,
#     amount_indices=AMOUNT_INDICES, 
#     price_diff_bins=PRICE_DIFF_BEST_BINS,
#     price_diff_indices=PRICE_DIFF_INDICES
# )

In [9]:
from src.model import OrderBookChunksCollection, OrderBook, S3OrderBookDataSource, OrderBooksDataSequenceDatasetV1
# s = time.time() 
order_books = source.get_all_order_books(keys=chunks[220].chunks(chunk_size=5000)[0].keys)
# print(time.time() -s)

In [10]:
import time 
s = time.time() 
dataset = OrderBooksDataSequenceDatasetV1(
    order_books, amount_best_bins=AMOUNT_BEST_BINS, amount_indices=AMOUNT_INDICES, price_diff_best_bins=PRICE_DIFF_BEST_BINS, price_diff_indices=PRICE_DIFF_INDICES
)
s = time.time() 
dataset.save("chunk4.tar.gz")
print(time.time() - s)

38.442007303237915


In [ ]:
OrderBooksDataSequenceDatasetV1.load("chunk4.tar.gz")

In [ ]:
from matplotlib import pyplot as plt 
plt.figure(figsize=(24,12))
pd.Series(Y[6,: ]).rolling(window=100).mean().plot()


In [ ]:
order_books[10].bids == order_books[0].bids

In [ ]:
import sys

In [ ]:
sys.getsizeof(dict(Y=Y, X=X))

In [12]:
dataset.validate()

True

In [ ]:
1634720444 - 1634719706

In [ ]:
from __future__ import annotations
import concurrent
import datetime
import traceback
import zlib
from abc import abstractmethod, ABC
from concurrent.futures import ThreadPoolExecutor
from functools import reduce
from multiprocessing import Pool

from typing import List, Dict

import dataclasses

import boto3
import msgpack
import numpy as np
import pandas as pd
from scipy import sparse

ACCEPTABLE_PRICE_DIFF = 0.03


In [ ]:
with open("/home/kamil/Downloads/1634719706", "rb") as f :
    object_content = f.read()
    decompressed_bytes = zlib.decompress(object_content)
    raw_dict = msgpack.unpackb(
        decompressed_bytes
    )

In [ ]:
x = raw_dict[b'asks']

In [ ]:
with open("/home/kamil/Downloads/1634720444", "rb") as f :
    object_content = f.read()
    decompressed_bytes = zlib.decompress(object_content)
    raw_dict = msgpack.unpackb(
        decompressed_bytes
    )

In [ ]:
raw_dict[b'asks'] == x

In [11]:
60 * 24


1440